<a href="https://colab.research.google.com/github/karasu1982/POS_Data_Analytics/blob/main/ABC%E5%88%86%E6%9E%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ABC分析

ABC分析は、商品の売上や在庫を分析するための第一歩といえる分析です。

売上を在庫金額を上位から並べていって、A・B・Cランクと分けていくことが基本的な流れです。

ランクを分ける基準としては、Aランクを売上上位70%（または80%）、Bランクでは同90%（Cランクは残り）と出すことが一般的です。

※ここでいう70%といった割合は商品数ではなく、「売上が上位の商品から並べて累積売上を出したときに、売上全体に占める割合」として出した数値です。


Aランクの商品数が、全体の商品数の何%なのかを見ることで、売上の大部分をどの程度の商品に頼っているのかを見ることができます。

パレートの法則では、よく２：８（または３：７）の法則といわれ、上位の２０％の商品で売上の８０％を占めるといわれています。

## 集計・分析のプログラムについての注意点

データ集計は、Pythonで行うパターンとSQLで行うパターンを、自分なりに最適と思う基準で分けています。

実践を考えたときに、多くのビジネスシーンで、元データ（今回はPOSデータ）がDWH（BigQueryやSnowflakeなどのクラウドも含めたSQLサーバ）に格納されている。それを、集計をするためにわざわざPythonでデータを取得し、Pandasに入れて集計するといったことが適さないケースが多いためです。

ただし、SQLのみで書いてしまうと自身でサーバを用意しないと、実行することができなくなってしまうため、[DuckDB](https://zenn.dev/paxdare_labo/articles/99c4e373a9fb58)を用いてSQLライクな実行を行っていきたいと思います。

## 環境設定

In [1]:
%%bash
pip install duckdb-engine

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.1/43.1 kB 1.4 MB/s eta 0:00:00


In [2]:
# 標準ライブラリ
import pandas as pd
import numpy as np

# DuckDB
import duckdb

## データ準備

ID-POSのサンプルデータとして、下記を利用

https://www.kyoritsu-pub.co.jp/book/b10003634.html

In [ ]:
%%bash
wget https://kyoritsu-pub.sakura.ne.jp/app/file/goods_contents/2319.zip
unzip /content/2319.zip
unzip /content/DataProcess.20151001/src/data/Tafeng/Tafeng.zip

In [4]:
df = pd.read_csv("/content/Tafeng_dataset/Tafeng.csv")

# データ型を
df = df.astype({'CustID': 'object', 'ProductSubClass': 'object', 'ProductID': 'object'})

## データ集計

In [5]:
import duckdb

# SQLに慣れている方にとっては、少々くどいかもしれませんが、可読性を高めるために、処理を１つ１つ分解してWITH句で実行しています。
#
# 私がBigQueryの文法に慣れていることもあり、次の3点で修正が必要でした。他にも微妙な差はありそうなので、都度都度直す必要がありますね。
# ・コメントアウトは#は使わず、-- で行う
# ・文字列はダブルコーテーション（ " )ではなく、シングルクオーテーション（ ' )でくくる
# ・日付項目は、文字列を自動で日付型化しないため、DATETIME '2000-11-01'のように日付型を明示的に指定する
#

df_output = duckdb.query(
    f"""
    WITH
    t_base AS(
      -- 商品コードごとの売上（単価×個数）の合計値を算出
      -- 期間を、2000年11～12月に絞る
      SELECT
        ProductID,
        SUM(SalesPrice * Amount) AS SalesTotal
      FROM df
      WHERE Time BETWEEN DATETIME '2000-11-01' AND DATETIME '2000-12-31'
      GROUP BY ProductID
    ),
    t_standard AS(
      -- 全体の売上のうち、70%を占める売上額・90%を占める売上額を算出
      SELECT
        SUM(SalesTotal) AS Sum_SalesTotal,
      FROM t_base
    ),
    t_cumulative AS(
      -- 売上を降順（高い順）でソートし、先頭からの累計売上額を算出
      SELECT
        ProductID,
        SalesTotal,
        SUM(SalesTotal) OVER (ORDER BY SalesTotal DESC) AS SalesCumulative
      FROM t_base
      ORDER BY SalesTotal DESC
    )

    SELECT
      ProductID,
      SalesTotal,
      SalesCumulative,
      SalesCumulative / Sum_SalesTotal AS Percentage_SalesCumulative,
      -- 累計売上額が売上合計の70%以下の場合はランクA、90%以下の場合はランクB、それ以降はランクCとしてランク付け
      CASE
        WHEN SalesCumulative / Sum_SalesTotal <= 0.7 THEN 'A'
        WHEN SalesCumulative / Sum_SalesTotal <= 0.9 THEN 'B'
        ELSE 'C'
      END AS SalesRank

    FROM t_cumulative
    FULL OUTER JOIN t_standard
    ON TRUE
    """
).to_df()

作成したテーブルのアウトプット
*   ProductID：商品コード。元々あって今回集計キーにした項目
*   SalesTotal：商品コード別の総売上
*   SalesCumulative：売上が高い順に並べたときの累積売上
*   Percentage_SalesCumulative：累積売上が、売上全体の何％を占めるのか
*   SalesRank：SalesRankで上位70%の売り上げを占める商品がA、90%を占める商品がB、それ以下がCとなっている

In [6]:
 # データ確認
df_output.head(100)

,ProductID,SalesTotal,SalesCumulative,Percentage_SalesCumulative,SalesRank
0,4710421090059,53326498.0,53326498.0,0.181332,A
1,8712045000151,33195168.0,86521666.0,0.294209,A
2,8712045008539,31166212.0,117687878.0,0.400187,A
3,4710628131012,19399462.0,137087340.0,0.466153,A
4,8712045011317,13260422.0,150347762.0,0.511244,A
...,...,...,...,...,...
95,4710043552102,132647.0,221091153.0,0.751801,B
96,4716821101203,129771.0,221220924.0,0.752242,B
97,20563356,127922.0,221348846.0,0.752677,B
98,4710205005750,125688.0,221474534.0,0.753105,B


In [7]:
# ランク別の商品数
df_output["SalesRank"].value_counts().sort_values()

A       29
B     1662
C    18587
Name: SalesRank, dtype: int64

売上全体の70%を占めるAランクの商品が、29商品（全体の1.4%）のみです。

よく70%の売り上げを20%や30%の商品で占めるというケースがありますが、このデータだと圧倒的に上位商品が売り上げに貢献していますね。